## 训练一两条样本

In [4]:
import torch
torch.cuda.is_available()

True

In [5]:
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [6]:
batch['labels'] = torch.tensor([1, 1])  # tokenizer出来的结果是一个dictionary，所以可以直接加入新的 key-value

optimizer = AdamW(model.parameters())
loss = model(**batch).loss  #这里的 loss 是直接根据 batch 中提供的 labels 来计算的，回忆：前面章节查看 model 的输出的时候，有loss这一项
loss.backward()
optimizer.step()

## 从Huggingface Hub中加载数据集

MRPC (Microsoft Research Paraphrase Corpus) dataset consists of 5,801 pairs of sentences, with a label indicating if they are paraphrases or not (i.e., if both sentences mean the same thing).

In [7]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Reusing dataset glue (C:\Users\Administrator\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

load_dataset出来的是一个DatasetDict对象，它包含了train，validation，test三个属性。可以通过key来直接查询，得到对应的数据集。

这里的train，valid，test都是Dataset类型，有 features和num_rows两个属性。还可以直接通过下标来查询对应的样本。

In [8]:
raw_train_dataset = raw_datasets['train']
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

Dataset的features可以理解为一张表的columns，Dataset甚至可以看做一个pandas的dataframe，二者的使用很类似。

我们可以直接像操作dataframe一样，取出某一列：

In [9]:
type(raw_train_dataset['sentence1'])  # 直接取出所有的sentence1，形成一个list

list

通过Dataset的features属性，可以详细查看数据集特征，包括labels具体都是啥：

In [10]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None),
 'idx': Value(dtype='int32', id=None)}

## 数据集的预处理

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

我们可以直接下面这样处理：
```python
tokenized_sentences_1 = tokenizer(raw_train_dataset['sentence1'])
tokenized_sentences_2 = tokenizer(raw_train_dataset['sentence2'])
```
但对于MRPC任务，我们不能把两个句子分开输入到模型中，二应该组成一个pair输进去。

tokenizer也可以直接处理sequence pair：

In [20]:
from pprint import pprint as print
inputs = tokenizer("first sentence", "second one")
print(inputs)

{'attention_mask': [1, 1, 1, 1, 1, 1, 1],
 'input_ids': [101, 2034, 6251, 102, 2117, 2028, 102],
 'token_type_ids': [0, 0, 0, 0, 1, 1, 1]}


In [21]:
tokenizer.decode(inputs.input_ids)

'[CLS] first sentence [SEP] second one [SEP]'

可以看到这里inputs里，还有一个`token_type_ids`属性，它在这里的作用就很明显了，指示哪些词是属于第一个句子，哪些词是属于第二个句子

这种神奇的做法，其实也是源于bert-base预训练的任务，即 next sentence prediction。换成其他模型，比如DistilBert，它在预训练的时候没有这个任务，那它的tokenizer的结果就不会有这个`token_type_ids`属性了。

既然这里的tokenizer可以直接处理pair，我们就可以这么去分词：

In [32]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

但是这样不一定好，因为它直接返回一个新的dictionary，会占据很多内存。

官方推荐的做法是通过`Dataset.map`方法，来调用一个分词方法，实现批量化的分词：

In [29]:
def tokenize_function(sample):
    # 这里可以添加多种操作，不光是tokenize
    return tokenizer(sample['sentence1'], sample['sentence2'], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

  0%|          | 0/4 [00:00<?, ?ba/s]

Loading cached processed dataset at C:\Users\Administrator\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-ef33b0c3c08e7836.arrow
Loading cached processed dataset at C:\Users\Administrator\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-aadac9a568777e3c.arrow


DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 408
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 1725
    })
})

看看这个map的一些参数：

```shell
raw_datasets.map(
    function,
    with_indices: bool = False,
    input_columns: Union[str, List[str], NoneType] = None,
    batched: bool = False,
    batch_size: Union[int, NoneType] = 1000,
    remove_columns: Union[str, List[str], NoneType] = None,
    keep_in_memory: bool = False,
    load_from_cache_file: bool = True,
    cache_file_names: Union[Dict[str, Union[str, NoneType]], NoneType] = None,
    writer_batch_size: Union[int, NoneType] = 1000,
    features: Union[datasets.features.Features, NoneType] = None,
    disable_nullable: bool = False,
    fn_kwargs: Union[dict, NoneType] = None,
    num_proc: Union[int, NoneType] = None,  # 使用此参数，可以使用多进程处理
    desc: Union[str, NoneType] = None,
) -> 'DatasetDict'
Docstring:
Apply a function to all the elements in the table (individually or in batches)
and update the table (if function does updated examples).
The transformation is applied to all the datasets of the dataset dictionary.
```

关于这个map：

What are the benefits of the Dataset.map method?
- The results of the function are cached, so it won't take any time if we re-execute the code.
    That is indeed one of the neat benefits of this method! It's not the only one, though...
- It can apply multiprocessing to go faster than applying the function on each element of the dataset.
    This is a neat feature of this method, but it's not the only one!
- It does not load the whole dataset into memory, saving the results as soon as one element is processed.
    That's one advantage of this method. There are others, though!

观察一下，这里通过map之后，得到的Dataset的features变多了：
```python
features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids']
```
多的几个columns就是tokenizer处理后的结果。

注意到，在这个`tokenize_function`中，我们没有使用`padding`，因为如果使用了padding之后，就会全局统一对一个maxlen进行padding，这样无论在tokenize还是模型的训练上都不够高效。

实际上，我们是故意先不进行padding的，因为我们想在划分batch的时候再进行padding。

**Question**：

但是我野看不到这里那里使用了batch内的padding，难道是默认的？以及不同batch如果有不同的maxlen，那训练的时候咋办？

这俩问题需要后面搞清楚！

## Dynamic Padding 动态padding

We have deliberately postponed the padding, to only apply it as necessary on each batch and avoid having over-long inputs with a lot of padding. This will speed up training by quite a bit.

In [34]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# 需要使用tokenizer来初始化
# to know which padding token to use, and whether the model expects padding to be on the left or on the right of the inputs

下面假设我们要搞一个size=5的batch，看看如何使用`DataCollatorWithPadding`来实现：

In [61]:
samples = tokenized_datasets['train'][:5]
samples.keys()
# ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids']
samples = {k:v for k,v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}  # 把这里多余的几列去掉
samples.keys()
# ['attention_mask', 'input_ids', 'label', 'token_type_ids']
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59]

In [62]:
batch = data_collator(samples)  # samples中必须包含 input_ids 字段，因为这就是collator要处理的对象
batch.keys()

dict_keys(['attention_mask', 'input_ids', 'token_type_ids', 'labels'])

In [57]:
[len(x) for x in batch['input_ids']]

[67, 67, 67, 67, 67]

可以看到，这个`data_collator`就是一个把给定dataset进行padding的工具，其输入跟输出是完全一样的格式。

In [64]:
{k:v.shape for k,v in batch.items()}

{'attention_mask': torch.Size([5, 67]),
 'input_ids': torch.Size([5, 67]),
 'token_type_ids': torch.Size([5, 67]),
 'labels': torch.Size([5])}

这个batch，可以形成一个tensor了！接下来就可以用于训练了！